## Proceso de Scrapping de Topicos


In [2]:
import asyncio
from twscrape import API, gather
from twscrape.logger import set_log_level
import pandas as pd
import datetime
import os
from dotenv import load_dotenv,dotenv_values


In [3]:
# creo api para hacer las llamadas
api = API()

### Defino Funciones

In [14]:
async def get_user_info(username):
    result = await api.user_by_login(username) 
    return result

In [15]:
async def get_user_tweets(user_id,limit=500):
    tweets = await gather(api.user_tweets(user_id,limit))
    return tweets

In [16]:
def parse_tweets(tweets,topic="",categoria="Tweet"):
    parsed_tweets = []
    for tweet in tweets:
            data = {
                    'Tweet id': tweet.id,
                    'Contenido': tweet.rawContent,
                    'Fecha': tweet.date,
                    'Categoria': categoria,
                    'Autor username': tweet.user.username,
                    'Autor displayname': tweet.user.displayname,
                    'Autor location': tweet.user.location,
                    'Link': tweet.url,
                    'Likes': tweet.likeCount,
                    'Respuestas': tweet.replyCount,
                    'Retweets': tweet.retweetCount,
                    'Hashtags': tweet.hashtags,
                    'Mentioned users': tweet.mentionedUsers,
                    'Topico':topic
                }
            parsed_tweets.append(data)
    return parsed_tweets

In [17]:
def parse_user(user):
    data = {
            'User id': user.id,
            'Autor username': user.username,
            'Autor displayname': user.displayname,
            'Autor location': user.location,
            'Fecha Creacion': user.created,
            'Link': user.url,
            'Verified': user.verified,
            'Followers': user.followersCount,
            'Following': user.friendsCount,
            'Tweets': user.statusesCount,
        }
    return data

In [18]:
def save_info(dataset,filename):
    df = pd.DataFrame(dataset)
    df.to_csv(filename, index=False)

In [19]:
def read_and_concat(ruta,data):
    df_new = pd.DataFrame(data)
    df = pd.read_csv(ruta)
    df_actualizado = pd.concat([df, df_new], ignore_index=True)
    df_actualizado.to_csv(ruta, index=False)

In [29]:
def try_read_and_concat(ruta,data):
    df_new = pd.DataFrame(data)
    try: 
        df = pd.read_csv(ruta)
        df_actualizado = pd.concat([df, df_new], ignore_index=True)
        df_actualizado.to_csv(ruta, index=False)
    except:
        df = pd.DataFrame(data)
        df.to_csv(ruta, index=False)
   

In [28]:
async def scrape_and_save(username,topic="",limit=500,ruta_tweets='data/topicos/tweets.csv',ruta_users='data/topicos/users.csv'):
    user_info = await get_user_info(username)
    tweets= await get_user_tweets(user_info.id,limit)
    print("Se obtuvieron " + str(len(tweets)) + " Tweets de la cuenta " + username)
    
    parsed_tweets = parse_tweets(tweets,topic)
    parsed_user = parse_user(user_info)
    print("Se parsearon correctamente los tweets")

    read_and_concat(ruta_tweets,parsed_tweets)
    read_and_concat(ruta_users,[parsed_user])
    print("Se almaceno correctamente")



In [30]:
async def try_scrape_and_save(username,ruta_tweets='data/topicos/tweets.csv',ruta_users='data/topicos/users.csv',topic="",limit=500):
    user_info = await get_user_info(username)
    tweets= await get_user_tweets(user_info.id,limit)
    print("Se obtuvieron " + str(len(tweets)) + " Tweets de la cuenta " + username)
    
    parsed_tweets = parse_tweets(tweets,topic)
    parsed_user = parse_user(user_info)
    print("Se parsearon correctamente los tweets")

    try_read_and_concat(ruta_tweets,parsed_tweets)
    try_read_and_concat(ruta_users,[parsed_user])
    print("Se almaceno correctamente")

### Obtengo tweets

In [41]:
 # Búsqueda de tweets por usuario:
username = "Politica_ln"
user_info = await get_user_info(username)
tweets_ln= await get_user_tweets(user_info.id,limit=1500)

In [42]:
len(tweets_ln)

850

In [98]:
for tweet in tweets_ln:
    print(tweet.rawContent)
    print("------------")

Patricia Bullrich será la ministra de Seguridad de Milei https://t.co/LhiEGHfPvG https://t.co/mhQXUpYFH4
------------
Graciela Camaño apuntó contra Randazzo: “Se va a tener que hacer cargo” https://t.co/ijFRcuSDWY https://t.co/4rCjT8LJwT
------------
Milei posterga su viaje a los Estados Unidos en medio de las turbulencias por el armado del gabinete https://t.co/Sd1VQYlTOD https://t.co/L657Lnl6Io
------------
La Corte dejó firme el cierre de la causa por el vaciamiento del Banco Mayo https://t.co/a5kZSa5Aw6 https://t.co/je1UsOIyen
------------
Jorge Macri define su gabinete con fuerte impronta Pro y la prioridad de finalizar el traspaso de competencias a la Ciudad https://t.co/q2ZMYK9sRw https://t.co/djdxpArXuZ
------------
Ante el temor de un “vaciamiento” de la coparticipación, los gobernadores peronistas se reúnen el martes https://t.co/yaKoANvmWI https://t.co/wgvX3YiLp3
------------
Anses: Píparo anunció que se reunirá con Raverta por la transición con un mensaje para “los trabajad

### Parseo los tweets

In [ ]:
parsed_ln = parse_tweets(tweets_ln,"Politica")
parsed_ln[1]

In [103]:
parsed_ln_user = parse_user(user_info)
parsed_ln_user

{'User id': 33936362,
 'Autor username': 'Politica_ln',
 'Autor displayname': 'LA NACION Política',
 'Autor location': 'Argentina',
 'Fecha Creacion': datetime.datetime(2009, 4, 21, 15, 26, 47, tzinfo=datetime.timezone.utc),
 'Link': 'https://twitter.com/Politica_ln',
 'Verified': False,
 'Followers': 210151,
 'Following': 121,
 'Tweets': 77762}

### Almaceno los tweets

In [124]:
# guardo todo en un dataframe para guardarlo
save_info(parsed_ln,"data/tweets1.csv")
save_info([parsed_ln_user],"data/users1.csv")


### Recopilo Tweets por Topico


#### Politica

In [18]:
await try_scrape_and_save("Politica_ln", topic="Politica",limit=2000)

Se obtuvieron 850 Tweets de la cuenta Politica_ln
Se parsearon correctamente los tweets
Se almaceno correctamente


In [35]:
await scrape_and_save("PoliticaClarin",topic="Politica",limit=1000)

Se obtuvieron 801 Tweets de la cuenta PoliticaClarin
Se parsearon correctamente los tweets
Se almaceno correctamente


In [45]:
await scrape_and_save("infobaepolitica",topic="Politica",limit=1500)

Se obtuvieron 849 Tweets de la cuenta infobaepolitica
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Deportes

In [134]:
await scrape_and_save("DiarioOle",topic="Deportes")

2023-11-23 23:49:35.010 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:260 - No account available for queue "UserTweets". Next available at 23:58:46
2023-11-23 23:58:51.791 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:266 - Account available for queue UserTweets


Se obtuvieron 517 Tweets de la cuenta DiarioOle
Se parsearon correctamente los tweets
Se almaceno correctamente


In [26]:
await scrape_and_save("DeportesLN",topic="Deportes",limit=1000)

Se obtuvieron 800 Tweets de la cuenta DeportesLN
Se parsearon correctamente los tweets
Se almaceno correctamente


In [46]:
await scrape_and_save("infobaedeportes",topic="Deportes",limit=1500)

2023-11-27 22:54:48.926 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:260 - No account available for queue "UserTweets". Next available at 22:57:15
2023-11-27 22:57:19.542 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:266 - Account available for queue UserTweets


Se obtuvieron 850 Tweets de la cuenta infobaedeportes
Se parsearon correctamente los tweets
Se almaceno correctamente


In [35]:
await scrape_and_save("elpais_deportes",topic="Deportes",limit=1500)

Se obtuvieron 851 Tweets de la cuenta elpais_deportes
Se parsearon correctamente los tweets
Se almaceno correctamente


In [23]:
await scrape_and_save("tndeportivo",topic="Deportes",limit=2000)

Se obtuvieron 853 Tweets de la cuenta tndeportivo
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Economia

In [19]:
await scrape_and_save("Economia_TN",topic="Economia",limit=2000)

Se obtuvieron 850 Tweets de la cuenta Economia_TN
Se parsearon correctamente los tweets
Se almaceno correctamente


In [44]:
await scrape_and_save("infobaeeconomia",topic="Economia",limit=1500)

Se obtuvieron 850 Tweets de la cuenta infobaeeconomia
Se parsearon correctamente los tweets
Se almaceno correctamente


In [34]:
await scrape_and_save("elpais_economia",topic="Economia",limit=1500)

Se obtuvieron 876 Tweets de la cuenta elpais_economia
Se parsearon correctamente los tweets
Se almaceno correctamente


In [31]:
await scrape_and_save("LNeconomia",topic="Economia",limit=1000)

2023-11-27 17:38:05.277 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:260 - No account available for queue "UserTweets". Next available at 17:38:44
2023-11-27 17:38:45.391 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:266 - Account available for queue UserTweets


Se obtuvieron 802 Tweets de la cuenta LNeconomia
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Entretenimiento

In [135]:
await scrape_and_save("PaparazziRevis",topic="Entretenimiento")


Se obtuvieron 521 Tweets de la cuenta PaparazziRevis
Se parsearon correctamente los tweets
Se almaceno correctamente


In [22]:
await scrape_and_save("TNFamosos",topic="Entretenimiento",limit=2000)

Se obtuvieron 850 Tweets de la cuenta TNFamosos
Se parsearon correctamente los tweets
Se almaceno correctamente


In [51]:
await scrape_and_save("teleshowcom",topic="Entretenimiento",limit=1500)

Se obtuvieron 852 Tweets de la cuenta teleshowcom
Se parsearon correctamente los tweets
Se almaceno correctamente


In [61]:
await scrape_and_save("ciudad_magazine",topic="Entretenimiento",limit=1500)

Se obtuvieron 850 Tweets de la cuenta ciudad_magazine
Se parsearon correctamente los tweets
Se almaceno correctamente


In [25]:
await scrape_and_save("PersonajesTV",topic="Entretenimiento",limit=1000)

Se obtuvieron 850 Tweets de la cuenta PersonajesTV
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Musica

In [20]:
await scrape_and_save("tnLaViola",topic="Musica",limit=2000)

Se obtuvieron 849 Tweets de la cuenta tnLaViola
Se parsearon correctamente los tweets
Se almaceno correctamente


In [36]:
await scrape_and_save("Los40ar",topic="Musica",limit=1500)

Se obtuvieron 1202 Tweets de la cuenta Los40ar
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Tecnologia


In [21]:
await scrape_and_save("TNTecno",topic="Tecnologia",limit=2000)

2023-11-25 17:00:57.203 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:260 - No account available for queue "UserTweets". Next available at 17:04:52
2023-11-25 17:04:57.833 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:266 - Account available for queue UserTweets


Se obtuvieron 851 Tweets de la cuenta TNTecno
Se parsearon correctamente los tweets
Se almaceno correctamente


In [27]:
await scrape_and_save("LNTecnologia",topic="Tecnologia",limit=1000)

2023-11-27 17:28:58.566 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:260 - No account available for queue "UserTweets". Next available at 17:29:07
2023-11-27 17:29:08.600 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:266 - Account available for queue UserTweets


Se obtuvieron 852 Tweets de la cuenta LNTecnologia
Se parsearon correctamente los tweets
Se almaceno correctamente


In [38]:
await scrape_and_save("elpais_tec",topic="Tecnologia",limit=1500)

2023-11-28 19:44:45.792 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:260 - No account available for queue "UserTweets". Next available at 19:47:58
2023-11-28 19:48:01.461 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:266 - Account available for queue UserTweets


Se obtuvieron 906 Tweets de la cuenta elpais_tec
Se parsearon correctamente los tweets
Se almaceno correctamente


In [64]:
await scrape_and_save("abc_tecnologia",topic="Tecnologia",limit=1500)

Se obtuvieron 850 Tweets de la cuenta abc_tecnologia
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Cine


In [21]:
await scrape_and_save("CINEMANIA_ES",topic="Cine",limit=1500)

Se obtuvieron 1134 Tweets de la cuenta CINEMANIA_ES
Se parsearon correctamente los tweets
Se almaceno correctamente


In [23]:
await scrape_and_save("fotogramas_es",topic="Cine",limit=1500)

Se obtuvieron 912 Tweets de la cuenta fotogramas_es
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Bienestar y Salud

In [45]:
await scrape_and_save("TNConBienestar",topic="Salud y Bienestar",limit=1500)

2023-11-28 19:53:11.696 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:260 - No account available for queue "UserTweets". Next available at 19:53:53
2023-11-28 19:53:56.897 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:266 - Account available for queue UserTweets


Se obtuvieron 850 Tweets de la cuenta TNConBienestar
Se parsearon correctamente los tweets
Se almaceno correctamente


In [59]:
await scrape_and_save("SaludeBienestar",topic="Salud y Bienestar",limit=1500)

Se obtuvieron 801 Tweets de la cuenta SaludeBienestar
Se parsearon correctamente los tweets
Se almaceno correctamente


In [46]:
await scrape_and_save("elpais_salud",topic="Salud y Bienestar",limit=1500)

Se obtuvieron 912 Tweets de la cuenta elpais_salud
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Arte y Cultura

In [55]:
await scrape_and_save("elpais_arte",topic="Arte y Cultura",limit=1500)

Se obtuvieron 801 Tweets de la cuenta elpais_arte
Se parsearon correctamente los tweets
Se almaceno correctamente


In [56]:
await scrape_and_save("elpais_cultura",topic="Arte y Cultura",limit=1500)

Se obtuvieron 851 Tweets de la cuenta elpais_cultura
Se parsearon correctamente los tweets
Se almaceno correctamente


### Analizo los tweets obtenidos por Topico

In [60]:
df = pd.read_csv("data/topicos/tweets.csv")
df.shape

(24405, 14)

In [40]:
df.head()

,Tweet id,Contenido,Fecha,Categoria,Autor username,Autor displayname,Autor location,Link,Likes,Respuestas,Retweets,Hashtags,Mentioned users,Topico
0,1728482459447353537,Máximo Kirchner aún no da señales al PJ bonaer...,2023-11-25 18:35:06+00:00,Tweet,Politica_ln,LA NACION Política,Argentina,https://twitter.com/Politica_ln/status/1728482...,9,1,1,[],[],Politica
1,1728418283370733667,Jorge Macri confirmó a los encargados de segur...,2023-11-25 14:20:05+00:00,Tweet,Politica_ln,LA NACION Política,Argentina,https://twitter.com/Politica_ln/status/1728418...,8,2,1,[],[],Politica
2,1728413500505276418,El papa Francisco envió un rosario bendecido a...,2023-11-25 14:01:05+00:00,Tweet,Politica_ln,LA NACION Política,Argentina,https://twitter.com/Politica_ln/status/1728413...,37,4,6,[],[],Politica
3,1728407462083231806,A tres años de su muerte: Cristina Kirchner ro...,2023-11-25 13:37:05+00:00,Tweet,Politica_ln,LA NACION Política,Argentina,https://twitter.com/Politica_ln/status/1728407...,2,0,1,[],[],Politica
4,1728385816597324110,El apuntado como nuevo titular del Conicet hab...,2023-11-25 12:11:05+00:00,Tweet,Politica_ln,LA NACION Política,Argentina,https://twitter.com/Politica_ln/status/1728385...,3,0,1,[],[],Politica


In [62]:
df_sin_duplicados = df.drop_duplicates(subset='Contenido', keep='first')
df_sin_duplicados.shape

(23679, 14)

In [63]:
df_sin_duplicados['Topico'].value_counts()

Topico
Tecnologia           3450
Entretenimiento      3402
Economia             3356
Deportes             3354
Politica             2500
Salud y Bienestar    2314
Musica               2010
Cine                 1648
Arte y Cultura       1645
Name: count, dtype: int64

In [61]:
df['Topico'].value_counts()

Topico
Tecnologia           3459
Entretenimiento      3402
Economia             3378
Deportes             3354
Salud y Bienestar    2563
Politica             2500
Musica               2051
Cine                 2046
Arte y Cultura       1652
Name: count, dtype: int64

In [50]:
autors = df['Autor username'].value_counts()
autors.head(n=20)


Autor username
CINEMANIA_ES       1067
fotogramas_es       903
elpais_tec          860
elpais_salud        856
LNTecnologia        852
Politica_ln         850
elpais_deportes     850
abc_tecnologia      850
ciudad_magazine     850
infobaedeportes     850
Los40ar             850
TNConBienestar      850
PersonajesTV        850
infobaeeconomia     850
tndeportivo         850
TNFamosos           850
Economia_TN         850
TNTecno             850
elpais_economia     850
infobaepolitica     849
Name: count, dtype: int64

### Obtengo info de personalidades

In [142]:
await try_scrape_and_save("tomasrebord","data/personalidades/rebord.csv","data/personalidades/rebord_info.csv", topic="HAGOV")

Se obtuvieron 933 Tweets de la cuenta tomasrebord
Se parsearon correctamente los tweets
Se almaceno correctamente


In [141]:
await try_scrape_and_save("DavooXeneizeJRR","data/personalidades/davo.csv","data/personalidades/davo_info.csv", topic="Boca")


Se obtuvieron 591 Tweets de la cuenta DavooXeneizeJRR
Se parsearon correctamente los tweets
Se almaceno correctamente


In [143]:
await try_scrape_and_save("CarlosMaslaton","data/personalidades/maslaton.csv","data/personalidades/maslaton_info.csv", topic="Proceda")

Se obtuvieron 645 Tweets de la cuenta CarlosMaslaton
Se parsearon correctamente los tweets
Se almaceno correctamente


### Tweets Massa y Milei


In [ ]:
tweets_milei = await gather(api.search("milei", limit=2000, kv={"product": "Top"}))

In [ ]:
# Hago búsqueda por un tipo de palabra y obtengo los que están en el top
tweets_milei =    await gather(api.search("milei", limit=2000, kv={"product": "Top"}))


In [ ]:
# Parseo los tweets obtenidos
tweets_milei_parseados = []
for tweet in tweets_milei:
        data_to_append = {
                'Titulo': tweet.id,
                'Contenido': tweet.rawContent,
                'Fecha': tweet.date,
                'Fuente': tweet.url,
                'Categoria': "Tweet",
                'Autor': tweet.user.username,
                'Link': tweet.url,
                'Clasificacion': "Tweet",
                'Narrativa':""
            }
        tweets_milei_parseados.append(data_to_append)

In [23]:
tweets_massa =    await gather(api.search("massa", limit=2000, kv={"product": "Top"}))

In [24]:
# Parseo los tweets obtenidos
tweets_massa_parseados = []
for tweet in tweets_massa:
        data_to_append = {
                'Titulo': tweet.id,
                'Contenido': tweet.rawContent,
                'Fecha': tweet.date,
                'Fuente': tweet.url,
                'Categoria': "Tweet",
                'Autor': tweet.user.username,
                'Link': tweet.url,
                'Clasificacion': "Tweet",
                'Narrativa':""
            }
        tweets_massa_parseados.append(data_to_append)

In [25]:
tweets_massa_parseados

[{'Titulo': 1726741230111834615,
  'Contenido': 'Sergio Chouza, a local keynesian economist very much in favor of the Alberto Fernandez government and Sergio Massa, deleted a tweet saying that Milei’s chances of getting elected were zero. https://t.co/TnGNrPjesA',
  'Fecha': datetime.datetime(2023, 11, 20, 23, 16, 5, tzinfo=datetime.timezone.utc),
  'Fuente': 'https://twitter.com/BowTiedMara/status/1726741230111834615',
  'Categoria': 'Tweet',
  'Autor': 'BowTiedMara',
  'Link': 'https://twitter.com/BowTiedMara/status/1726741230111834615',
  'Clasificacion': 'Tweet',
  'Narrativa': ''},
 {'Titulo': 1726733482599039085,
  'Contenido': "New map from me:\n\nYesterday anarcho-capitalist Javier Milei defeated Peronist Sergio Massa to become Argentina's next president,\n\nMilei saw his largest margins in the central provinces of Córdoba &amp; Mendoza, while Massa's best province by far was Santiago del Estero. https://t.co/vZ63VeCHPs",
  'Fecha': datetime.datetime(2023, 11, 20, 22, 45, 18, t

In [ ]:
# guardo todo en un dataframe para guardarlo
df = pd.DataFrame()
df = pd.concat([pd.DataFrame(tweets_milei_parseados), pd.DataFrame(tweets_massa_parseados)], ignore_index=True)
df.to_csv("tweets_scrappeados.csv", index=False)